### week7 심화과제

RAG 서비스를 제공하는 솔루션에 있는 문서에 따른 질문 추천 모델을 파인튜닝

In [ ]:
import json

with open("raw_data/KorQuAD_v1.0_dev.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [38]:
print(f"데이터 개수 : {len(data['data'])}")
print(data['data'][30])

데이터 개수 : 140
{'paragraphs': [{'qas': [{'answers': [{'text': 'TBC', 'answer_start': 208}], 'id': '5782847-0-0', 'question': 'jtbc가 설립하기전에 처음 사용하려고 했던 이름은 무엇인가?'}, {'answers': [{'text': '2009년 7월 22일', 'answer_start': 0}], 'id': '5782847-0-1', 'question': '종편채널 설립에 있어 법적 근거를 부여하는 방송법이 국회를 통과된 날은 언제인가?'}, {'answers': [{'text': '대구방송', 'answer_start': 275}], 'id': '5782847-0-2', 'question': '이름을 jtbc전에 tbc를 사용하려 했으나 이름이 선점하고 있었기 때문인데, 어느 곳이 tbc라는 이름을 사용하고 있었나?'}, {'answers': [{'text': '2010년 11월 30일', 'answer_start': 147}], 'id': '6489801-0-0', 'question': '종합편성채널의 신청서가 접수된 해는?'}, {'answers': [{'text': 'TBC', 'answer_start': 208}], 'id': '6489801-0-1', 'question': 'JTBC 전 종합편성채널에 신청하려 했건 이름은?'}, {'answers': [{'text': 'TBC', 'answer_start': 208}], 'id': '6470395-0-0', 'question': '중앙일보가 종합편성채널의 명칭으로 사용하고자 했던 이름은 무엇인가?'}, {'answers': [{'text': '동양방송', 'answer_start': 195}], 'id': '6470395-0-1', 'question': '중앙일보의 종합편성채널 JTBC라는 명칭에서 TBC는 무엇의 영문 약칭인가?'}], 'context': '2009년 7월 22일에 여러 차례 논란 끝에 

#### ✍️ 학습용 데이터 생성

이미 korQuAD는 QA dataset이기 때문에 해당 데이터에 있는 question을 활용하여 LLM을 사용하지 않고 데이터셋을 만들 수 있습니다.   
RAG에 들어오는 chunk 마다 추천 질문을 만들 것 이기 때문에 여러 질문을 생성하게 하지 않고 한개의 질문을 생성하게 합니다.   
  - 1개의 문서 -> 20개의 chunk -> chunk당 추천 질문 생성 -> 20개의 추천 질문 생성됨 (random 하게 n개 제공)   

In [43]:
import random

instruction_data = []

for doc in data["data"]:
    paragraphs = doc["paragraphs"]
    valid_paragraphs = [p for p in paragraphs if p["qas"]]  # 질문이 있는 문단만

    if not valid_paragraphs:
        continue  # 질문 없는 문서 제외

    # 랜덤으로 하나 선택
    para = random.choice(valid_paragraphs)
    context = para["context"].strip()
    question = para["qas"][0]["question"].strip()

    # instruction-format 구성
    instruction_data.append({
        "instruction": "다음 문단을 읽고 질문을 만들어 주세요.",
        "input": context,
        "output": question if question.endswith("?") else question + "?"
    })

# 결과 예시 출력
print(f"총 {len(instruction_data)}개 생성됨")
print(json.dumps(instruction_data[0], ensure_ascii=False, indent=2))

총 140개 생성됨
{
  "instruction": "다음 문단을 읽고 질문을 만들어 주세요.",
  "input": "\"내각과 장관들이 소외되고 대통령비서실의 권한이 너무 크다\", \"행보가 비서 본연의 역할을 벗어난다\"는 의견이 제기되었다. 대표적인 예가 10차 개헌안 발표이다. 원로 헌법학자인 허영 경희대 석좌교수는 정부의 헌법개정안 준비 과정에 대해 \"청와대 비서실이 아닌 국무회의 중심으로 이뤄졌어야 했다\"고 지적했다. '국무회의의 심의를 거쳐야 한다'(제89조)는 헌법 규정에 충실하지 않았다는 것이다. 그러면서 \"법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다\"고 지적했다. 민정수석은 국회의원에 대해 책임지는 법무부 장관도 아니고, 국민에 대해 책임지는 사람도 아니기 때문에 정당성이 없고, 단지 대통령의 신임이 있을 뿐이라는 것이다. 또한 국무총리 선출 방식에 대한 기자의 질문에 \"문 대통령도 취임 전에 국무총리에게 실질적 권한을 주겠다고 했지만 그러지 못하고 있다. 대통령비서실장만도 못한 권한을 행사하고 있다.\"고 답변했다.",
  "output": "정부의 헌법개정안 준비 과정에 대해서 청와대 비서실이 아니라 국무회의 중심으로 이뤄졌어야 했다고 지적한 원로 헌법학자는?"
}


In [44]:
with open("corpus.json", "w", encoding="utf-8") as f:
    json.dump(instruction_data, f, ensure_ascii=False, indent=2)